[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [28]:
import numpy as np
import pandas as pd
import math
from common import *

norm = np.linalg.norm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

### [Wrapper za brojanje fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

In [2]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        f.called = 0
        try:
            return fn(f, *args, **kwargs)
        finally:
            wrapper.calls_to_function = f.called
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije cilja

In [3]:
@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [4]:
@counts
def golden_section(f, a, b, e=1e-6, k=0.5*(math.sqrt(5) + 1)):
    hi = b - (b - a) / k
    lo = a + (b - a) / k
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while abs(b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - (b - a) / k
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + (b - a) / k
            f_hi = f_lo
            f_lo = f(lo)
    
    return (a + b) / 2

In [5]:
@counted 
def test_gs_fn(x): 
    return (x - 4.99)**2

print(golden_section(test_gs_fn, 0, 5))
print(golden_section.calls_to_function)

4.989999842904119
35


### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

Trazi unimodalni dio funkcije

In [6]:
@counts
def unimodal(f, pt, h=1):
    l, r = pt - h, pt + h
    m = pt
    step = 1
    
    fm = f(pt)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while True:
            l, m, fm = m, r, fr
            r += h * step
            fr = f(r)
            step *= 2
            if fm <= fr:
                return l, r
    else:
        while True:
            r, m, fm = m, l, fl
            l -= h * step
            fl = f(l)
            step *= 2
            if fm <= fl:
                return l, r
    

In [38]:
unimodal(f1, pt(1,1))

(array([ 0.,  0.]), array([ 2.,  2.]))

### Coordinate descent
TODO uncopy

In [7]:
@counts
def coordinate_descent(f, x0, eps=1e-6):
    x = np.array(x0, dtype=float)
    x_prev = x + 2 * eps
    while norm(x_prev - x) > eps:
        x_prev = np.copy(x)
        for i in range(len(x)):
            def fi(xt):
                xp = np.copy(x)
                xp[i] = xt
                return f(xp)
            l, r = unimodal(fi, 0.1, x[i])
            x[i] = golden_section(fi, l, r, eps)
    return x

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [8]:
class InvalidAlgorithmSetupException(Exception):
    def __init__(self, message):
        super(InvalidAlgorithmSetupException, self).__init__(message)
    
    def raiseIf(condition, message = "Invalid algorithm setup"):
        if condition:
            raise InvalidAlgorithmSetupException(message)

In [9]:
@counts
def nelder_mead(
    f, x0, step=1, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6, max_iter=1000):
    InvalidAlgorithmSetupException.raiseIf(alpha <= 0)
    InvalidAlgorithmSetupException.raiseIf(gamma <= 1)
    InvalidAlgorithmSetupException.raiseIf(sigma <= 0 or sigma > 0.5)
    
    x = simplex_pts(x0, step)
    l, h = 0, -1
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    expand = lambda xc, xr: xc + gamma * (xr - xc)
    contract = lambda xc, xh: xc - beta * (xc - xh)
    shrink = lambda xl, xi: xl + sigma * (xi - xl)
    
    i = 0
    while True:
        i += 1
        
        x = sorted(x, key=f)
        xc = np.mean(x[:h], axis=0)
        
        if norm(x[h] - xc) <= epsilon or i >= max_iter:
            break
        
        xr = reflect(xc, x[h])
        
        fxr = f(xr)
        fxl = f(x[l])
        
        if fxr < fxl:
            xe = expand(xc, x[h])
            x[h] = xe if f(xe) < fxl else xr
        else:
            if all(fxr > f(xj) for xj in x[:h]):
                if fxr < f(x[h]):
                    x[h] = xr
                    
                xk = contract(xc, x[h])
                if f(xk) < f(x[h]):
                    x[h] = xk
                else:
                    shrink_xi = lambda xi: shrink(x[l], xi)
                    x = lmap(shrink_xi, x)
            else:
                x[h] = xr
            
    return x[0]

def simplex_pts(x0, step):
    ret = [x0]
    for i in range(len(x0)):
        xp = np.copy(x0)
        xp[i] += step
        ret.append(xp)
    return np.array(ret)
        

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [10]:
@counts
def hooke_jeeves(f, x0, epsilon=1e-6, dx=0.5):
    dx = dx * np.ones_like(x0)
    epsilon = epsilon * np.ones_like(x0)

    xp = np.copy(x0)
    xb = np.copy(x0)

    while all(dx > epsilon):
        xn = explore(f, xp, dx)
        if f(xn) < f(xb):
            xp = 2 * xn - xb
            xb = np.copy(xn)
        else:
            dx /= 2.0
            xp = np.copy(xb)
        
    return xp

def explore(f, xp, dx):
    x = np.array(xp, dtype=float)
    P = f(x)
    for i in range(len(xp)):
        x[i] = float(x[i]) + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

# Zadaci
## 1.
Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [13]:
data = []

for x0 in [ pt(10), pt(20), pt(30) ]:
    for f in [ hooke_jeeves, nelder_mead, coordinate_descent ]:
        result = f(f3, x0)
        data.append([f.__name__, x0, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,[10.0],132,[1.0]
1,nelder_mead,[10.0],207,[1.0]
2,coordinate_descent,[10.0],77,[1.00000003808]
3,hooke_jeeves,[20.0],147,[1.0]
4,nelder_mead,[20.0],257,[1.0]
5,coordinate_descent,[20.0],79,[0.999999998747]
6,hooke_jeeves,[30.0],158,[1.0]
7,nelder_mead,[30.0],307,[1.0]
8,coordinate_descent,[30.0],80,[0.999999803366]


## 2.
Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za
svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata?

In [14]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]

data = []
for search_f in [hooke_jeeves, nelder_mead]:
    for f, x0 in zip(fs, x0s):
        result = search_f(f, x0)
        data.append(
            [search_f.__name__, x0, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,"[-1.9, 2.0]",2952,"[1.00000152588, 1.0000038147]"
1,hooke_jeeves,"[0.1, 0.3]",260,"[3.99999961853, 2.00000076294]"
2,hooke_jeeves,"[0.0, 0.0, 0.0, 0.0, 0.0]",342,"[1.0, 2.0, 3.0, 4.0, 5.0]"
3,hooke_jeeves,"[5.1, 1.1]",150,"[3.1, 3.1]"
4,nelder_mead,"[-1.9, 2.0]",3562,"[0.999999763998, 0.999999487937]"
5,nelder_mead,"[0.1, 0.3]",493,"[3.99999995768, 1.99999972735]"
6,nelder_mead,"[0.0, 0.0, 0.0, 0.0, 0.0]",2774,"[0.999999846924, 1.99999971796, 3.00000003086, 4.00000002405, 4.99999947917]"
7,nelder_mead,"[5.1, 1.1]",712,"[-8.3331812619e-07, 1.51895016504e-07]"


## 3.
Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate!

In [15]:
hooke_jeeves(f4, pt(5, 5))

array([ 5.,  5.])

In [16]:
nelder_mead(f4, pt(5, 5))

array([ -8.84859430e-08,  -3.86806220e-07])

## 4.
Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

Zakljucujem da se algoritam puno bolje ponasa kada za pocetnu tocku uzmemo neku koja je blizu stvarnog minimuma, i to neovisno o koraku koji odaberemo.

In [31]:
data = []
for x0 in [ pt(0.5, 0.5), pt(20, 20) ]:
    for step in range(1, 21):
        result = nelder_mead(f1, x0, step=step)
        data.append([x0, step, nelder_mead.calls_to_function, result, f1(result)])
        
pd.DataFrame(data, columns=['x0', 'Step', 'Num calls', 'Result', 'Cost'])

,x0,Korak,Broj poziva,Rezultat,Cost
0,"[0.5, 0.5]",1,418,"[1.00000663406, 1.00001278068]",6.777450e-11
1,"[0.5, 0.5]",2,4677,"[0.99999989005, 0.999999747216]",1.202295e-13
2,"[0.5, 0.5]",3,2878,"[1.00000161348, 1.00000318316]",2.795237e-12
3,"[0.5, 0.5]",4,1672,"[0.999999272771, 0.999998478111]",9.835678e-13
4,"[0.5, 0.5]",5,1237,"[1.00000047531, 1.00000095745]",2.305811e-13
5,"[0.5, 0.5]",6,6418,"[0.736152049736, 0.542237324233]",6.962582e-02
6,"[0.5, 0.5]",7,861,"[0.999999873029, 0.999999707377]",1.657410e-13
7,"[0.5, 0.5]",8,1312,"[1.00000088168, 1.00000171537]",1.007768e-12
8,"[0.5, 0.5]",9,1011,"[0.99999903608, 0.99999809265]",9.711259e-13
9,"[0.5, 0.5]",10,707,"[0.999999067494, 0.999998073962]",1.241989e-12


## 5.
Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

In [37]:
ncalls = 200
threshold = 1e-4

data = []
for i in range(ncalls):
    random = np.random.rand(2) * 50
    result = hooke_jeeves(f6, pt(*random))
    cost = f6(result)
    
    data.append([random,
                 hooke_jeeves.calls_to_function, 
                 result,
                 cost,
                 cost < threshold ])

df = pd.DataFrame(data, columns=['x0', 'Num calls', 'Result', 'Cost', 'Min'])
print('Nasao globalni optimum:', np.any(df['Min']))
df

False


,x0,Num calls,Result,Cost,Min
0,"[10.8210822773, 46.421014555]",204,"[10.5139362049, 45.921014555]",0.451776,False
1,"[42.7193817122, 30.3774941802]",221,"[43.0868190749, 31.5334943175]",0.466295,False
2,"[45.5942743099, 35.6093401028]",217,"[34.0789678372, 45.1093401028]",0.471615,False
3,"[47.0781594239, 27.9246526695]",303,"[41.8822213135, 33.1166845299]",0.466295,False
4,"[11.6457422609, 11.0009262647]",173,"[10.1154573793, 12.0009262647]",0.178222,False
5,"[41.4046876226, 5.9552890526]",193,"[40.4046876226, 5.84589688646]",0.429723,False
6,"[4.16534508809, 14.5106272296]",180,"[4.5853890334, 15.0106272296]",0.178222,False
7,"[19.9565202315, 43.9076532176]",180,"[19.447769316, 42.9076532176]",0.451776,False
8,"[46.3420725769, 49.348301545]",200,"[47.3316336579, 50.348301545]",0.485013,False
9,"[28.6285485934, 42.7706497818]",216,"[28.6691426943, 41.2706497818]",0.459781,False


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)